In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/Users/kazi0302/Desktop/SDSU/MSCSDS/COVID-19-Modeling-and-Parameter-Fitting/Data/Local Mobility Data.xlsx - HHSA.csv')

In [3]:
central = 511660
east = 488330
n_central = 646380
n_coastal = 535219
n_inland = 602000
south = 500076

In [4]:
for index, row in df.iterrows():
    if type(row['Updated May 8 Number']) is str:
        df.at[index, 'Updated May 8 Number'] = row['Updated May 8 Number'].replace(',', '')

In [5]:
Central = {}
East = {}
N_Central = {}
N_Coastal = {}
N_Inland = {}
South = {}


for index, row in df.iterrows():
    if row['h_hhsa_name (ORIGIN)'] == 'CENTRAL' and row['w_hhsa_name (DESTINATION)'] not in Central:
        Central[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/central]
    elif row['h_hhsa_name (ORIGIN)'] == 'EAST' and row['w_hhsa_name (DESTINATION)'] not in East:
        East[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/east]
    elif row['h_hhsa_name (ORIGIN)'] == 'NORTH CENTRAL' and row['w_hhsa_name (DESTINATION)'] not in N_Central:
        N_Central[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/n_central]
    elif row['h_hhsa_name (ORIGIN)'] == 'NORTH COASTAL' and row['w_hhsa_name (DESTINATION)'] not in N_Coastal:
        N_Coastal[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/n_coastal]
    elif row['h_hhsa_name (ORIGIN)'] == 'NORTH INLAND' and row['w_hhsa_name (DESTINATION)'] not in N_Inland:
        N_Inland[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/n_inland]
    elif row['h_hhsa_name (ORIGIN)'] == 'SOUTH' and row['w_hhsa_name (DESTINATION)'] not in South:
        South[row['w_hhsa_name (DESTINATION)']] = [int(row['Baseline All Jobs']), int(row['Updated May 8 Number']), int(row['Updated May 8 Number'])/south]

Central.popitem()
East.popitem()
N_Central.popitem()
N_Coastal.popitem()
N_Inland.popitem()
South.popitem()

Central = pd.DataFrame.from_dict(Central, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])
East = pd.DataFrame.from_dict(East, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])
N_Central = pd.DataFrame.from_dict(N_Central, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])
N_Coastal = pd.DataFrame.from_dict(N_Coastal, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])
N_Inland = pd.DataFrame.from_dict(N_Inland, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])
South = pd.DataFrame.from_dict(South, orient='index', columns=['Before 5/8', 'After 5/8', 'ratio moved'])

In [6]:
print('##########Central##########')
print(Central)
print('\n##########East##########')
print(East)
print('\n##########North Central##########')
print(N_Central)
print('\n##########North Coastal##########')
print(N_Coastal)
print('\n##########North Inland##########')
print(N_Inland)
print('\n##########South##########')
print(South)

##########Central##########
               Before 5/8  After 5/8  ratio moved
NORTH COASTAL        6265       6265     0.012244
NORTH CENTRAL       77369      47969     0.093752
CENTRAL             52925      67744     0.132400
SOUTH               19312      12939     0.025288
EAST                18068      12286     0.024012
NORTH INLAND         9451       9451     0.018471

##########East##########
               Before 5/8  After 5/8  ratio moved
NORTH COASTAL        6217       6217     0.012731
NORTH CENTRAL       61336      73603     0.150724
CENTRAL             27209      24488     0.050146
SOUTH               13998      13998     0.028665
EAST                62893      83019     0.170006
NORTH INLAND        12511      12511     0.025620

##########North Central##########
               Before 5/8  After 5/8  ratio moved
NORTH COASTAL       13229      13229     0.020466
NORTH CENTRAL      135269     169086     0.261589
CENTRAL             34489      24487     0.037883
SOUTH      